<a href="https://colab.research.google.com/github/hantonio/imersaoai-alura-google/blob/main/Buscador_de_Vagas_no_LinkedIn_e_Gerador_de_Recomenda%C3%A7%C3%B5es.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Buscador de vagas de emprego no LinkedIn e Gerador de Recomendações para Iniciantes

O projeto busca fornecer um programa que permita a busca de vagas no LinkedIn através de uma palavra ou termo que descreva o cargo (por exemplo, "DevOps") e uma localização (por exemplo, "Brazil").

A partir dessas entradas o programa busca por esses termos na API do LinkedIn, fornecendo uma lista de vagas, bem como recomendações de estudo para transição de carreira ou aperfeiçoamento de habilidades, e algumas características da empresa que está contratando. Ao final ele fornece dicas para ir bem na entrevista de emprego.

## Instalação de dependências

In [16]:
!pip install -U -q google-generativeai numpy pandas==2.0.3 beautifulsoup4

## Importação de dependências

In [17]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import google.generativeai as genai
from timeit import default_timer as timer
from google.colab import userdata

## Definição de variáveis

In [18]:
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)
job_entries_list = []

keyword="DevOps"
location="Brazil"

url = f'https://www.linkedin.com/jobs/search?keywords={keyword}&location={location}&pageNum=0'
model = "models/text-embedding-004"
generation_config = {
  "temperature": 0,
  "candidate_count": 1
}

model_2 = genai.GenerativeModel("gemini-1.0-pro",
                               generation_config=generation_config)

## Preparar dataframe com vagas coletadas do LinkedIn

In [19]:
response = requests.get(url)
if response.status_code == 200:
   soup = BeautifulSoup(response.text, 'html.parser')
   job_listings = soup.find_all('div', {'class':'job-search-card'})
   for job in job_listings:
        title = job.find('h3', {'class': 'base-search-card__title'}).text.strip()
        company = job.find('a', {'class': 'hidden-nested-link'}).text.strip()
        location = job.find('span', {'class': 'job-search-card__location'}).text.strip()
        anchor_tag = job.find('a', class_='base-card__full-link')
        href_link = anchor_tag['href']
        details = ""
        response_details = requests.get(href_link)
        if response_details.status_code == 200:
            soup2 = BeautifulSoup(response_details.text, 'html.parser')
            job_posting = soup2.find('div', {'class': 'show-more-less-html__markup show-more-less-html__markup--clamp-after-5 relative overflow-hidden'}).text.strip()
        job_entry = { "title" : title, "company": company, "location": location, "link": href_link, "description": job_posting}
        job_entries_list.append(job_entry)
else:
   print("Failed to fetch job listings.")

df = pd.DataFrame(job_entries_list)
df.columns = ["title", "company", "location", "link", "description"]
df = df.head(5) #limitando para 5 entradas por limitação de timeout do Google Colab

## Definição de funções para embedding e sumarização

In [20]:
def embed_fn(title, description):
   return genai.embed_content(model=model, content=description, title=title, task_type="RETRIEVAL_DOCUMENT")["embedding"]

def job_summarizer(description):
    prompt = f"Resuma o texto de descrição fornecido neste trecho focando especificamente na vaga de emprego, no nome ou título da vaga oferecida e nos requisitos profissionais: {description}"
    response = model_2.generate_content(prompt)
    return response.text

def company_summarizer(description):
    prompt = f"Resuma o texto de descrição fornecido neste trecho focando especificamente na empresa que oferece a vaga de emprego e no nome ou título da vaga oferecida: {description}"
    response = model_2.generate_content(prompt)
    return response.text

## Sumarização e embedding das descrições de vagas e companhias

In [21]:
# Gerar coluna de sumarização de descrição da vaga
start = timer()
df["shortdescription"] = df.apply(lambda row: job_summarizer(row["description"]), axis=1)
end = timer()
print(end - start)

18.85165299800002


In [22]:
# Gerar coluna de sumarização de descrição da empresa
start = timer()
df["companydescription"] = df.apply(lambda row: company_summarizer(row["description"]), axis=1)
end = timer()
print(end - start)

11.112911124999982


In [23]:
# Geração de embeddings para vagas
start = timer()
df["embeddings"] = df.apply(lambda row: embed_fn(row["title"], row["shortdescription"]), axis=1)
end = timer()
print(end - start)

5.0851910800000155


In [24]:
# Geração de embeddings para empresas
start = timer()
df["embeddings_c"] = df.apply(lambda row: embed_fn(row["title"], row["companydescription"]), axis=1)
end = timer()
print(end - start)

4.9224091680000015


## Definição de funções de geração de prompt e consulta

In [25]:
def gerar_e_buscar_consulta(consulta, base, model):
  embedding_da_consulta = genai.embed_content(model=model,
                                 content=consulta,
                                 task_type="RETRIEVAL_QUERY")["embedding"]

  produtos_escalares = np.dot(np.stack(df["embeddings"]), embedding_da_consulta)

  indice = np.argmax(produtos_escalares)
  return df.iloc[indice]["shortdescription"]

def gerar_e_buscar_consulta2(consulta, base, model):
  embedding_da_consulta = genai.embed_content(model=model,
                                 content=consulta,
                                 task_type="RETRIEVAL_QUERY")["embedding"]

  produtos_escalares = np.dot(np.stack(df["embeddings_c"]), embedding_da_consulta)

  indice = np.argmax(produtos_escalares)
  return df.iloc[indice]["companydescription"]

## Vagas encontradas

In [26]:
print(df[['title','companydescription','shortdescription']])

                              title  \
0        Analista de Sistemas Pleno   
1  [RPE] Analista CloudOps - Júnior   
2               Analista DevOps Jr.   
3         PL Desenvolvimento Python   
4                 Estágio em DevOps   

                                  companydescription  \
0      **Empresa:** Alelo\n\n**Vaga:** Líder Técnico   
1  **Empresa:** RPE - Retail Payment Ecosystem\n\...   
2  **Empresa:** UOL EdTech\n\n**Vaga:** DevOps En...   
3  **Empresa:** ília\n\n**Vaga:** Desenvolvedor P...   
4  **Empresa:** dti digital\n\n**Vaga:** Estagiár...   

                                    shortdescription  
0  **Vaga de Emprego:** Líder Técnico\n\n**Requis...  
1  **Vaga de Emprego:** Analista de TechOps\n\n**...  
2  **Vaga de Emprego:** Engenheiro de DevOps\n\n*...  
3  **Vaga de Emprego:** Desenvolvedor PYTHON\n\n*...  
4  **Vaga de Emprego:** Estagiário DevOps\n\n**Re...  


## Consultas a base de vagas

### Quais são as habilidades mais requisitadas de um profissional de acordo com a vaga no LinkedIn?"

In [30]:
consulta = "Quais são as habilidades requisitadas para um Estagiário?"
trecho = gerar_e_buscar_consulta(consulta, df, model)
print(f"Resultado de busca: {trecho}")

Resultado de busca: **Vaga de Emprego:** Estagiário DevOps

**Requisitos Profissionais:**

**Requisitos Básicos:**

* Inclinação ou interesse em DevOps
* Inglês avançado
* Matrícula em curso superior, técnico ou ensino médio
* Curso em Engenharia de Controle e Automação, Engenharia de Sistemas, Engenharia Elétrica, Ciência da Computação, Engenharia da Computação ou Sistemas de Informação

**Requisitos Desejáveis:**

* Conhecimento em C#, .NET
* Conhecimento em desenvolvimento Java/Angular
* Conhecimento em desenvolvimento Front End (HTML, JS, CSS)
* Conhecimento em desenvolvimento mobile para iOS e Android


### Sugira um plano de estudo para um iniciante que está migrando de carreira

In [31]:
prompt = f"Baseado nesse {trecho}, gere um plano de estudos para alguém que está migrando de carreira."
response = model_2.generate_content(prompt)
print(response.text)

**Plano de Estudos para Migração de Carreira para Estagiário DevOps**

**Objetivo:** Adquirir os conhecimentos e habilidades essenciais para se qualificar para uma vaga de Estagiário DevOps.

**Duração:** 6-12 meses

**Requisitos Prévios:**

* Inclinação ou interesse em DevOps
* Inglês avançado
* Matrícula em curso superior, técnico ou ensino médio

**Módulos do Curso:**

**Módulo 1: Fundamentos de DevOps**

* Conceitos e princípios de DevOps
* Metodologias ágeis (Scrum, Kanban)
* Ferramentas de automação (Jenkins, Ansible)
* Gerenciamento de configuração (Puppet, Chef)

**Módulo 2: Desenvolvimento de Software**

* Linguagens de programação (C#, Java)
* Desenvolvimento Front End (HTML, JS, CSS)
* Desenvolvimento mobile (iOS, Android)

**Módulo 3: Infraestrutura de TI**

* Sistemas operacionais (Linux, Windows)
* Virtualização (VMware, Docker)
* Redes e segurança
* Armazenamento e backup

**Módulo 4: Ferramentas e Tecnologias DevOps**

* Git e controle de versão
* Integração contínua (C

In [34]:
consulta2 = "Qual é a empresa que está oferecendo a vaga de Estagiario?"
trecho2 = gerar_e_buscar_consulta2(consulta2, df, model)
print(f"Resultado de busca: {trecho2}")

Resultado de busca: **Empresa:** dti digital

**Vaga:** Estagiário DevOps


### Quais vagas são adequadas para um iniciante?

In [35]:
prompt2 = f"Conte um pouco sobre a empresa que está oferecendo a vaga: {trecho2}"
response2 = model_2.generate_content(prompt2)
print(response2.text)

**Sobre a dti digital**

A dti digital é uma empresa brasileira de tecnologia da informação (TI) fundada em 2002. Com sede em São Paulo, a empresa oferece uma ampla gama de serviços de TI, incluindo:

* Desenvolvimento e manutenção de software
* Infraestrutura de TI
* Segurança cibernética
* Consultoria de TI
* Treinamento e certificação

A dti digital tem uma forte presença no mercado financeiro, atendendo a grandes bancos e instituições financeiras. A empresa também atua em outros setores, como varejo, saúde e educação.

A dti digital é reconhecida por sua inovação e excelência em TI. A empresa recebeu vários prêmios e certificações, incluindo:

* Great Place to Work
* ISO 9001:2015
* CMMI Nível 3

A dti digital está comprometida em fornecer soluções de TI de alta qualidade que atendam às necessidades específicas de seus clientes. A empresa investe fortemente em pesquisa e desenvolvimento para se manter na vanguarda das tendências tecnológicas.


### Recomendações para entrevista de emprego

In [37]:
prompt3 = f"Dado os trechos a seguir que descrevem a vaga e a empresa, sugira dicas para ir bem em uma entrevista de emprego: {trecho} e {trecho2}"
response3 = model_2.generate_content(prompt3)
print(response3.text)

**Dicas para Ir Bem em uma Entrevista de Emprego para Estagiário DevOps**

**Antes da Entrevista:**

* **Pesquise a empresa:** Conheça a dti digital, sua cultura e seus valores.
* **Revise os requisitos da vaga:** Certifique-se de entender os requisitos básicos e desejáveis.
* **Prepare exemplos:** Pense em exemplos específicos de projetos ou experiências que demonstrem suas habilidades em DevOps.
* **Pratique suas respostas:** Ensaie suas respostas para perguntas comuns de entrevista, como "Conte-me sobre você" e "Por que você está interessado nesta vaga?".

**Durante a Entrevista:**

* **Seja profissional:** Vista-se adequadamente e chegue na hora.
* **Demonstre entusiasmo:** Mostre que você está genuinamente interessado na vaga e na empresa.
* **Destaque suas habilidades:** Enfatize suas habilidades em DevOps, mesmo que sejam limitadas.
* **Faça perguntas:** Faça perguntas perspicazes sobre a vaga, a equipe e a cultura da empresa.
* **Seja curioso:** Pergunte sobre as ferramentas e 